# Generate trajectories

## Import packages

In [1]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.polynomial import legendre
from scipy.linalg import block_diag

from pyrotor.constraints import is_in_constraints
from pyrotor.projection import trajectory_to_coef, coef_to_trajectory
from pyrotor.data_analysis import compute_covariance
from pyrotor.linear_conditions import get_endpoints_matrix

## Define functions

In [2]:
def order_of_magnitude(x):
    """
    Find order of magnitude of each component of an array.
    """
    alpha = np.floor(np.log10(np.abs(x)))
    
    return np.nan_to_num(alpha)

In [3]:
def projection_kernel(basis_dimension, endpoints):
    """
    Compute projector onto the kernel of the matrix phi describing endpoints conditions.
    """
    # Build endpoints conditions matrix
    phi = get_endpoints_matrix(basis_dimension, endpoints)
    # Compute SVD
    _, S, V = np.linalg.svd(phi, full_matrices=True)
    # Find singular vectors in kernel
    indices_kernel = np.where(S == 0)
    if len(indices_kernel[0]) > 0:
        first_index = indices_kernel[0][0]
    else:
        first_index = len(S)
    # Compute projector
    V = V.T
    P_kerphi = np.dot(V[:,first_index:], V[:,first_index:].T)
    
    return P_kerphi

## Initialise generation

### Define constraints

In [4]:
# x1 > 0
def f1(data):
    x1 = data["x1"].values
    return x1

# x1 < 1
def f2(data):
    x1 = data["x1"].values
    return 1 - x1

# x2 > 0
def f3(data):
    x2 = data["x2"].values
    return x2

# x2 < 1
def f4(data):
    x2 = data["x2"].values
    return 1 - x2

# x2 > f(x1)
def f5(data):
    x1 = data["x1"].values
    x2 = data["x2"].values
    return x2 - 150/19 * (1-x1)**3 + 225/19 * (1-x1)**2 - 100/19 * (1-x1) + 79/190 - .05

constraints = [f1, f2, f3, f4, f5]

### Define initial and final states

In [5]:
example = 2

In [6]:
if example == 1:
    endpoints = {'x1': {'start': .1,
                        'end': .9,
                        'delta': .01},
                 'x2': {'start': .828,
                        'end': .172,
                        'delta': .01}}
elif example == 2:
    endpoints = {'x1': {'start': .1,
                        'end': .9,
                        'delta': 0},
                 'x2': {'start': .9,
                        'end': .2,
                        'delta': 0}}

### Define independent variable (time)

In [7]:
independent_variable = {'start': .1,
                        'end': .9,
                        'frequency': .01}
# Compute number of evaluation points
delta_time = independent_variable['end'] - independent_variable['start']
delta_time /= independent_variable['frequency']
independent_variable['points_nb'] = int(delta_time) + 1

### Define reference trajectory

In [8]:
# First component
def y1(t):
    if example == 1:
        return t
    elif example == 2:
        if .1 <= t < .5:
            y1 = .75 * (t - .1) + .1
        elif .5 <= t <= .9:
            y1 = 1.25 * (t - .9) + .9
        return y1
# Second component
def y2(t):
    if example == 1:
        return -2 * t**3 + 3 * t**2 - 2 * t + 1
    elif example == 2:
        if .1 <= t < .3:
            y2 = -3 * t + 1.2
        elif .3 <= t < .7:
            y2 = .25 * t + .225
        elif .7 <= t <= .9:
            y2 = -t + 1.1
        return y2

# Create dataframe
y = pd.DataFrame()
time = np.linspace(independent_variable['start'],
                   independent_variable['end'],
                   independent_variable['points_nb'])
y['x1'] = np.array([y1(t) for t in time])
y['x2'] = np.array([y2(t) for t in time])

### Plot to visualise reference trajectory and constraints

In [9]:
X = np.linspace(0, 1, 101)
constraint_f5 = np.array([150/19 * (1-x)**3 - 225/19 * (1-x)**2 + 100/19 * (1-x) - 79/190 for x in X])

fig, ax = plt.subplots(figsize=(10,7))
ax.plot(y['x1'], y['x2'], label='Reference trajectory', color='b')
ax.fill_between(X, 0, constraint_f5, color='r', alpha=.5, label='Forbidden area')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_xlim(left=0, right=1)
ax.set_ylim(bottom=0, top=1)
ax.legend()
plt.tight_layout()

### Define functional basis and project reference trajectory

In [10]:
# Basis name
basis = 'legendre'
# Dimension for each variable
basis_dimension = {'x1': 4,
                   'x2': 6}

In [11]:
# Project
c = trajectory_to_coef(y, basis, basis_dimension)

### Compute magnitude of each coefficient and add up small perturbations

In [12]:
magnitude = []
# Compute magnitude
magnitude = order_of_magnitude(c)
# Add Gaussian noise
noise = np.random.normal(0, 1, len(c))
magnitude += noise.astype(int)

## Generate trajectories

### Compute projector over phi kernel to preserve endpoints conditions

In [13]:
P_kerphi = projection_kernel(basis_dimension, endpoints)

### Generate new trajectories via perturbation

In [14]:
# Choose number of flights to generate
I = 1000
# Generate
coefs_reference = []
for i in range(I):
    # Generate Gaussian noise depending on order of magnitude of coefficients
    noise = np.random.normal(0, .2, len(c)) * np.float_power(10, magnitude)
    # Project noise onto kernel of phi
    noise = np.dot(P_kerphi, noise)
    # Perturbe
    coef_reference = c + noise
    coefs_reference.append(coef_reference)

### Build trajectories

In [15]:
trajs_reference = []
points_nb = len(y)
for i in range(I):
    yi = coef_to_trajectory(coefs_reference[i], points_nb, 'legendre', basis_dimension)
    trajs_reference.append(yi)

### Check constraints and keep acceptable generated trajectories

In [16]:
trajs_acceptable = []
cost_by_time = np.zeros(points_nb)
for i in range(I):
    boolean = is_in_constraints(trajs_reference[i], constraints, cost_by_time)
    if boolean:
        trajs_acceptable.append(trajs_reference[i].drop(columns='cost'))

trajs_acceptable_nb = len(trajs_acceptable)
print('Number of acceptable trajectories = ', trajs_acceptable_nb)

### Plot

In [17]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(y['x1'], y['x2'], label='Reference trajectory', color='b')
for i in range(trajs_acceptable_nb):
    ax.plot(trajs_acceptable[i]['x1'], trajs_acceptable[i]['x2'], label='_nolegend_', linestyle='--')
ax.fill_between(X, 0, constraint_f5, color='r', alpha=.5, label='Forbidden area')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_xlim(left=0, right=1)
ax.set_ylim(bottom=0, top=1)
ax.legend()
plt.tight_layout()
plt.savefig('fig.svg')

### Export to csv files

In [18]:
# Create folder
#now = datetime.now()
#dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
#path = 'generated_trajectories_' + dt_string
#os.mkdir(path)
# Save generated trajectories
#for i in range(trajs_acceptable_nb):
#    trajs_acceptable[i].to_csv(path + '/trajectory_' + str(i) + '.csv')